# Intro to Pandas

What is Pandas?

![Alt text](http://getwallpapers.com/wallpaper/full/1/2/3/1185666-cute-anime-panda-wallpaper-1920x1200-hd.jpg) 

Pandas is a data structures, analysis, and manipulation tool. 

The name comes from the term '**pan**el **da**ta', a term used in statistics to describe multi-dimensional, tabulated data.

What can you do with pandas?

![Alt text](images/Pandas.png)

### 1) Import Package

In [ ]:
import pandas 

Importing as pd makes it easier to call:

In [ ]:
import pandas as pd

Import other package(s)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

### 2) Series

- indexed one-dimensional array

In [ ]:
data = pd.Series([0, 10, 100, 1000])
data

### 3) DataFrames

- two-dimensional array

In [ ]:
df = pd.DataFrame([[1, 2], [3, 4], [5, 6], [7, 8]],columns = ['odds', 'evens'])
df

Or, you can create a datafame with multiple series.

In [ ]:
tens = pd.Series([0, 10, 20, 30])
hundreds = pd.Series([0, 100, 200, 300])

df = pd.DataFrame({'tens':tens, 'hundreds': hundreds})
df

Try creating your own dataframe:

### 4) Import Data from file

We can also create dataframes by importing data from files. This is *extremely* valuable, as we can now analyze and manipulate large datasets quickly.

File types include:
* csv
* text
* JSON
* html
* Excel
* hdf5
* SQL
* And many others!

Let's take a look at a file from a battery cycling experiment. This file is the output from an Arbin battery cycler. 

The data is from Sandia National Labs: https://www.sandia.gov/energystoragesafety-ssl/research-development/research-data-repository/



First, import the file.

In [ ]:
data = pd.read_excel('data/LFP_1_25C_Reg.xls', sheet_name = 'Channel_1-001')

`data.head()` and `data.tail()` allow you to see the beginning and end of the dataframe respectively. The automatic output is 5 rows. Place a number in the pararentheses to change the number of rows.

In [ ]:
data.head()

In [ ]:
data.head(2)

In [ ]:
data.tail()

In [ ]:
data.tail(10)

What does the first 15 rows look like? The last 20?

`len()` shows you the length of the dataframe

In [ ]:
len(data)

Use `data.loc[]` or `data.iloc[]` to call a specific row. `.loc` uses an index begininning with 1, and `.iloc` uses an index beginning with 0.

In [ ]:
data.loc[100]

In [ ]:
data.iloc[99]

You can extract a certain column from the dataframe. The output is a series. 

In [ ]:
data['Current(A)']

In [ ]:
data['Current(A)'].head()

In [ ]:
data['Current(A)'].values

In [ ]:
data[1:10]

What is the current measurement for the 152nd row?

You can change the index of the dataframe:

In [ ]:
data.index

In [ ]:
data = data.set_index('Data_Point')

In [ ]:
data.head()

In [ ]:
data.columns

You can perform operations between columns and place the result in a new column:

In [ ]:
data['avg_temp'] = (data['Temperature (C)_1'] + data['Temperature (C)_2'] + data['Temperature (C)_3'] + data['Temperature (C)_4'])/4
data.head()

Create a new dataframe with a condition statement.

In [ ]:
data_step = data[data['Step_Index'] == 43]
data_step.head()

What information would you like to extract from the data? Try it finding it here:

### 5) Calculate Capacity 

How many charge/discharge cycles were used for this experiment?

In [ ]:
data['Cycle_Index'].max()

One way to find capacity is to use a list and for loop:

In [ ]:
capacity_charge = []
capacity_discharge = []
for i in range(1,int(data['Cycle_Index'].max())+1):
    cycle = data[data['Cycle_Index'] == i]
    charge = cycle['Charge_Capacity(Ah)'].max()
    discharge = cycle['Discharge_Capacity(Ah)'].max()
    capacity_charge.append(charge)
    capacity_discharge.append(discharge)
    
capacity_discharge = pd.DataFrame(capacity_discharge, columns = ['Discharge Capacity (Ah)'])
capacity_discharge

However, Pandas gives us a much easier and faster way to accomplish the same task, using `df.groupby`

In [ ]:
cycles = data.groupby('Cycle_Index')
capacity_discharge = cycles['Discharge_Capacity(Ah)'].max()
capacity_discharge

In [ ]:
capacity_discharge_single = capacity_discharge[2]
cell_capacity = pd.DataFrame([[file, capacity_discharge_single]])
cell_capacity

### 6) Save as a new file

In [ ]:
capacity_discharge.to_csv('capacity_discharge.csv')

### 7) Plot results

In [ ]:
plt.plot(data['Test_Time(s)'],data['Voltage(V)'])

### 8) Iterate through files

What makes Pandas extra powerful, is we can iterate through multiple files and perform the same analysis. 

In [ ]:
from os import listdir

In [ ]:
cell_capacities = pd.DataFrame()

for file in listdir('./data'):
    data = pd.read_excel('./data/%s' % file, sheet_name = 1)
    cycles = data.groupby('Cycle_Index')
    capacity_discharge_all = cycles['Discharge_Capacity(Ah)'].max()
    capacity_discharge = capacity_discharge_all[2]
    cell_capacity = pd.DataFrame([[file, capacity_discharge]], columns = ['File Name', 'Capacity'])
    cell_capacities = cell_capacities.append(cell_capacity)

In [ ]:
cell_capacities

Resources used to create this tutorial:
* http://pandas.pydata.org/pandas-docs/stable/
* https://jakevdp.github.io/PythonDataScienceHandbook/03.01-introducing-pandas-objects.html
* https://www.youtube.com/playlist?list=PLQVvvaa0QuDfSfqQuee6K8opKtZsh7sA9
* https://www.dlr.de/sc/Portaldata/15/Resources/dokumente/pyhpc2011/submissions/pyhpc2011_submission_9.pdf